# Experiments for Identifying Buried Water Sites

&nbsp;

### Preparation
I'd like everything to be easily visualizable, so let's work in 2D. Admittedly, this will lower the complexity of possible cavity arrangements, but hopefully will serve as a sufficient sandbox to attempt various approaches to the site-identification problem.



-------------------------------

In [ ]:
## imports & configuration

# standard imports
import json

# custom imports
import ipywidgets
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import scipy.spatial
import tqdm.notebook

# inline plots
# %config InlineBackend.figure_formats = ['svg']
%matplotlib notebook

# jupyter theme
try:
    import jupyterthemes as jt
    jt.jtplot.style()
except ImportError:
    pass

-------------------------------

In [ ]:
## class to represent drawable matplotlib canvas
class Canvas:
    def __init__(self):
        pass

    def init_figure(self, figsize=(5, 3)):
        self.fig, self.ax = plt.subplots(figsize=figsize)
        for s in ['top','bottom','left','right']:
            self.ax.spines[s].set_linewidth(2)
        self.ax.set_aspect('equal', 'box')
        self.ax.set_xlim(0, 1)
        self.ax.set_ylim(0, 1)
        self.ax.xaxis.set_ticks([])
        self.ax.yaxis.set_ticks([])
        self.fig.tight_layout()

    def draw_circles(self, positions, radii, style=None):
        # set default circle style
        style = style or dict(edgecolor="C0", linewidth=2, fill=None)

        # if radius is a scalar, convert to array
        if isinstance(radii, (int, float)):
            radii = radii*np.ones(len(positions))

        # loop through particles
        return [
            self.ax.add_patch(matplotlib.patches.Circle(xy=p, radius=r, **style))
            for i, (p, r) in enumerate(zip(positions, radii))
        ]

In [ ]:
## custom slider ipywidget class
class Slider:
    # build widget objects
    def __init__(self, update_method, index_var, **kwargs):
        # play button
        defaults = dict(value=0, min=0, max=99, step=1, interval=10, disabled=False)
        defaults.update(kwargs)
        self.play = play = ipywidgets.Play(**defaults)

        # make interactive slider
        self.int_slider = ipywidgets.IntSlider(min=play.min, max=play.max, step=play.step, value=play.value)
        self.slider = ipywidgets.interactive(update_method, **{index_var: self.int_slider})

        # link play button to slider value
        ipywidgets.jslink((self.play, 'value'), (self.slider.children[0], 'value'))

        # construct player widget
        self.player = ipywidgets.HBox([self.play, self.slider])

In [ ]:
## trajectory configuration

# trajectory length
n_frames = 100

# set box size (scaling)
box_size = 50 # Å

In [ ]:
## construct fake protein trajectory

# construct grid of hexagonally-spaced points
def construct_hex_grid(m, n):
    tzip2d = lambda xs, ys: np.concatenate(np.stack(np.meshgrid(xs, ys)).T, axis=0)
    return np.concatenate([
        1+tzip2d(2*np.sqrt(3)*(0+1*np.arange(m//2)), 0+2*np.arange(n//2)),
        1+tzip2d(1*np.sqrt(3)*(1+2*np.arange(m//2)), 1+2*np.arange(n//2)),
    ], axis=0)

# construct protein grid
grid_shape = (32, 56) # rows and columns
margin = 0.4 # fraction of grid radius
n_prot_base = np.prod(grid_shape)
l_max = max(grid_shape[0]*np.sqrt(3), grid_shape[1])
grid_radius = 1/(l_max)
scale = 1/(l_max+1)
grid_pos = scale*(np.array([0.5, 0])+construct_hex_grid(*grid_shape))
prot_radius = (1 - margin)*grid_radius

# propagate grid positions through vector field
field_func = lambda x: np.sin(10*(x-0.5))[:, ::-1]
flow_positions = np.zeros((n_frames, len(grid_pos), 2))
flow_positions[0] = grid_pos
for i in range(1, n_frames):
    flow_positions[i] = flow_positions[i-1]+0.001*field_func(flow_positions[i-1])
    flow_positions[i] = (0.999*(flow_positions[i]-0.5))+0.5

# construct random perturbations
perts = np.random.randn(n_frames, len(grid_pos), 2)
perts /= np.linalg.norm(perts, axis=-1)[:, :, None]
perts *= margin*grid_radius*np.random.random((n_frames, len(grid_pos)))[:, :, None]

# use SDF to subselect from grid
def smin(a, b, k=32):
    res = np.exp2(-k * a) + np.exp2(-k * b)
    return np.log2(res) / -k
sub = lambda x, y, **kwargs: -smin(-x, y, **kwargs)
SDF = lambda x: sub(
    sub(
        sub(
            sub(
                np.linalg.norm(x - np.array([0.5, 0.5]), axis=-1)-0.4,
                np.linalg.norm(x - np.array([0.3, 0.3]), axis=-1)-0.04,
            ),
            np.linalg.norm(x - np.array([0.28, 0.52]), axis=-1)-0.1,
        ),
        smin(
            smin(
                np.linalg.norm(x - np.array([0.5, 0.8]), axis=-1)-0.01,
                np.linalg.norm(x - np.array([0.6, 0.7]), axis=-1)-0.005,
                k=16
            ),
            smin(
                np.linalg.norm(x - np.array([0.7, 0.6]), axis=-1)-0.005,
                np.linalg.norm(x - np.array([0.8, 0.5]), axis=-1)-0.01,
                k=16
            ),
            k=16
        )
    ),
    smin(
        np.linalg.norm(x - np.array([0.6, 0.3]), axis=-1)-0.01,
        np.linalg.norm(x - np.array([0.6, 0.2]), axis=-1)-0.01,
        k=16
    )
)
prot_sel = SDF(grid_pos) < 0.

# output final protein positions
prot_pos = (flow_positions+perts)[:, prot_sel]

In [ ]:
## construct water trajectories

# compute water-box size at standard state concentration
# (55.345 mol/L * 6.022e23 w/mol 1e3 L/m3 1e-30 m3/Å3) == 0.033328759 w/Å3
s_wat = (1/(55.345 * 6.022e23 * 1e3 * 1e-30))**(1/3) # or 3.10737465 Å / water

# choose a size for water (water radius is 1.925 pm)
r_wat = 1.925*0.5 # Å

# set the number of particles and radii
n_waters = int((box_size**2)/(s_wat**2))

# maximum # of packing attempts per water
n_attempts = 10000

# initialize container
wat_pos = np.nan*np.ones((n_frames, n_waters, 2))

# iterate through frames
for i in tqdm.notebook.tqdm(range(n_frames)):
    # loop through waters
    for j in range(n_waters):
        # try n_attempt times to pack
        for k in range(n_attempts):
            # random position without bounds
            pos = r_wat+(box_size-2*r_wat)*np.random.random(2)

            # compute displacements
            disp = wat_pos[i][:j] - pos[None]

            # if no collisions, add position and break
            if np.all((disp*disp).sum(axis=-1) > (2*r_wat)**2):
                wat_pos[i][j] = pos
                break

        # error if too many attempts without packing!
        if k+1 >= n_attempts:
            raise RuntimeError(f"frame {i} water {j} couldn't be packed with {n_attempts} attempts")

# scale waters
wat_pos /= box_size
wat_radius = r_wat/box_size

# resolve water-protein collisions
collisions = np.any(np.linalg.norm(prot_pos[:, None] - wat_pos[:, :, None], axis=-1) < prot_radius+wat_radius, axis=-1)
wat_pos[collisions] *= 0

In [ ]:
## draw interactive canvas
canvas = Canvas()
canvas.init_figure(figsize=(7, 7))
circles = []
circles += canvas.draw_circles(wat_pos[0], wat_radius)
circles += canvas.draw_circles(prot_pos[0], prot_radius, style=dict(edgecolor="k", facecolor="#C0C", linewidth=2))
positions = np.concatenate([wat_pos, prot_pos], axis=1)
def update(frame):
    for i, circle in enumerate(circles):
        circle.center = positions[frame][i]
Slider(update, "frame", max=len(positions)-1).player

In [ ]:
## visualize protein selection SDF

# evalaute SDF over grid
X, Y = np.meshgrid(np.linspace(0, 1, 50), np.linspace(0, 1, 50))
Z = SDF(np.concatenate(np.stack([Y, X]).T)).reshape(X.shape)

# view SDF
plt.figure()
plt.title("SDF")
plt.contourf(X, Y, Z, levels=10)
plt.colorbar()
plt.tight_layout()

In [ ]:
## visualize distortion vector field

# compute vector field
pts = np.concatenate(np.stack(np.meshgrid(np.linspace(0, 1, 30), np.linspace(0, 1, 30)), axis=0).T)

# view vector field
plt.figure()
plt.title("$f(x, y) = (sin(y), sin(x))$")
plt.quiver(*pts.T, *field_func(pts).T)
plt.tight_layout()

-------------------------

##### Brownian Motion
Let's start with some simple molecular motion. This [Git repo](https://github.com/xnx/collision/blob/master/collision.py) has a numpy/scipy-based implementation of MD with hard-spheres. That should be a good starting point.

## UPDATE: THIS WAS A VERITABLY BAD IDEA

In [ ]:
# d_mat = scipy.spatial.distance_matrix(wat_pos, wat_pos)
# tri_indices = np.triu_indices(len(wat_pos), k=+1)
# ds = d_mat[tri_indices]

In [ ]:
## MD code (github.com/xnx/collision/blob/master/collision.py)

# imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib import animation
from itertools import combinations

# particle class
class Particle:
    """A class representing a two-dimensional particle."""

    def __init__(self, x, y, vx, vy, radius=0.01):
        """Initialize the particle's position, velocity, and radius.
        """

        self.r = np.array((x, y))
        self.v = np.array((vx, vy))
        self.radius = radius
        self.mass = self.radius**2

    # For convenience, map the components of the particle's position and
    # velocity vector onto the attributes x, y, vx and vy.
    @property
    def x(self):
        return self.r[0]
    @x.setter
    def x(self, value):
        self.r[0] = value
    @property
    def y(self):
        return self.r[1]
    @y.setter
    def y(self, value):
        self.r[1] = value
    @property
    def vx(self):
        return self.v[0]
    @vx.setter
    def vx(self, value):
        self.v[0] = value
    @property
    def vy(self):
        return self.v[1]
    @vy.setter
    def vy(self, value):
        self.v[1] = value

    def overlaps(self, other):
        """Does the circle of this Particle overlap that of other?"""

        return np.hypot(*(self.r - other.r)) < self.radius + other.radius

    def advance(self, dt):
        """Advance the Particle's position forward in time by dt."""

        self.r += self.v * dt

# simulation class
class Simulation:
    """A class for a simple hard-circle molecular dynamics simulation.
    The simulation is carried out on a square domain: 0 <= x < 1, 0 <= y < 1.
    """

    ParticleClass = Particle

    def __init__(self, n, dt=0.01, radius=0.01):
        """Initialize the simulation with n Particles with radii radius.
        radius can be a single value or a sequence with n values.
        """

        self.init_particles(n, radius)
        self.dt = dt

    def place_particle(self, rad):
        # Choose x, y so that the Particle is entirely inside the
        # domain of the simulation.
        x, y = rad + (1 - 2*rad) * np.random.random(2)
        # Choose a random velocity (within some reasonable range of
        # values) for the Particle.
        vr = 0.1 * np.sqrt(np.random.random()) + 0.05
        vphi = 2*np.pi * np.random.random()
        vx, vy = vr * np.cos(vphi), vr * np.sin(vphi)
        particle = self.ParticleClass(x, y, vx, vy, rad)
        # Check that the Particle doesn't overlap one that's already
        # been placed.
        for p2 in self.particles:
            if p2.overlaps(particle):
                break
        else:
            self.particles.append(particle)
            return True
        return False

    def init_particles(self, n, radius, styles=None):
        """Initialize the n Particles of the simulation.
        Positions and velocities are chosen randomly; radius can be a single
        value or a sequence with n values.
        """

        try:
            iterator = iter(radius)
            assert n == len(radius)
        except TypeError:
            # r isn't iterable: turn it into a generator that returns the
            # same value n times.
            def r_gen(n, radius):
                for i in range(n):
                    yield radius
            radius = r_gen(n, radius)

        self.n = n
        self.particles = []
        for i, rad in enumerate(radius):
            # Try to find a random initial position for this particle.
            while not self.place_particle(rad):
                pass

    def change_velocities(self, p1, p2):
        """
        Particles p1 and p2 have collided elastically: update their
        velocities.
        """
        
        m1, m2 = p1.mass, p2.mass
        M = m1 + m2
        r1, r2 = p1.r, p2.r
        d = np.linalg.norm(r1 - r2)**2
        v1, v2 = p1.v, p2.v
        u1 = v1 - 2*m2 / M * np.dot(v1-v2, r1-r2) / d * (r1 - r2)
        u2 = v2 - 2*m1 / M * np.dot(v2-v1, r2-r1) / d * (r2 - r1)
        p1.v = u1
        p2.v = u2

    def handle_collisions(self):
        """Detect and handle any collisions between the Particles.
        When two Particles collide, they do so elastically: their velocities
        change such that both energy and momentum are conserved.
        """ 

        # We're going to need a sequence of all of the pairs of particles when
        # we are detecting collisions. combinations generates pairs of indexes
        # into the self.particles list of Particles on the fly.
        pairs = combinations(range(self.n), 2)
        for i,j in pairs:
            if self.particles[i].overlaps(self.particles[j]):
                self.change_velocities(self.particles[i], self.particles[j])

    def handle_boundary_collisions(self, p):
        """Bounce the particles off the walls elastically."""

        if p.x - p.radius < 0:
            p.x = p.radius
            p.vx = -p.vx
        if p.x + p.radius > 1:
            p.x = 1-p.radius
            p.vx = -p.vx
        if p.y - p.radius < 0:
            p.y = p.radius
            p.vy = -p.vy
        if p.y + p.radius > 1:
            p.y = 1-p.radius
            p.vy = -p.vy

    def apply_forces(self):
        """Override this method to accelerate the particles."""
        pass

    def advance(self):
        """Advance the animation by dt."""
        for i, p in enumerate(self.particles):
            p.advance(self.dt)
            self.handle_boundary_collisions(p)
        self.handle_collisions()
        self.apply_forces()

# animation class
class Animation:
    def __init__(self, simulation, positions, styles=None):
        """Any key-value pairs passed in the styles dictionary will be passed
        as arguments to Matplotlib's Circle patch constructor.
        """

        self.simulation = simulation
        self.positions = positions
        self.styles = styles
        self.setup_animation()
        self.init()

    def setup_animation(self):
        self.fig, self.ax = plt.subplots()
        for s in ['top','bottom','left','right']:
            self.ax.spines[s].set_linewidth(2)
        self.ax.set_aspect('equal', 'box')
        self.ax.set_xlim(0, 1)
        self.ax.set_ylim(0, 1)
        self.ax.xaxis.set_ticks([])
        self.ax.yaxis.set_ticks([])
#         plt.close(self.fig)

    def init(self):
        """Initialize the Matplotlib animation."""
        self.circles = []
        for particle in self.simulation.particles:
            self.circles.append(self.draw(particle))
        return self.circles

    def draw(self, particle):
        """Add this Particle's Circle patch to the Matplotlib Axes ax."""

        # default circle styles
        styles = self.styles or {'edgecolor': 'b', 'fill': False}

        circle = Circle(xy=particle.r, radius=particle.radius, **styles)
        self.ax.add_patch(circle)
        return circle

    def set_frame(self, t):
        for i, p in enumerate(self.simulation.particles):
            self.circles[i].center = self.positions[t][i]
        [c.remove() for c in self.ax.patches]
        [self.ax.add_patch(c) for c in self.circles]

In [ ]:
## configure sim

# set box size (scaling)
box_size = 50 # Å

# compute water-box size at standard state concentration
# (55.345 mol/L * 6.022e23 w/mol 1e3 L/m3 1e-30 m3/Å3) == 0.033328759 w/Å3
s_wat = (1/(55.345 * 6.022e23 * 1e3 * 1e-30))**(1/3) # or 3.10737465 Å / water

# choose a size for water (water radius is 1.925 pm, drop a dimension)
r_wat = 1.925/s_wat # Å

# set the number of particles and radii
n_particles = int((box_size**2)/(s_wat**2))
radii = np.ones_like(n_particles)*r_wat/box_size

In [ ]:
# initialize sim (packs particles)
sim = Simulation(n_particles, dt=0.1, radius=radii)

In [ ]:
# run sim
positions = np.stack(list(zip(*[
    (sim.advance(), np.stack([p.r for p in sim.particles]))
    for i in tqdm.notebook.tqdm(range(100))
]))[1])

In [ ]:
## prepare animation and player

# animation
animation = Animation(sim, positions, {'edgecolor': 'C0', 'linewidth': 2, 'fill': None})

## prepare ipywidget

# play button
play = ipywidgets.Play(value=0, min=0, max=len(positions)-1, step=1, interval=10, disabled=False)

# make interactive slider
slider = ipywidgets.IntSlider(min=play.min, max=play.max, step=play.step, value=play.value)
slider = ipywidgets.interactive(animation.set_frame, t=slider)

# link play button to slider value
ipywidgets.jslink((play, 'value'), (slider.children[0], 'value'))

# construct player widget
player = ipywidgets.HBox([play, slider])

## show both
# plt.figure(animation.fig)
player

-------------------------------

-------------------------------